In [1]:
import os
import boto3
import json
from botocore.exceptions import ClientError
from dotenv import load_dotenv
load_dotenv()

AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
YOUR_SECRET_ACCESS_KEY=os.getenv("AWS_ACCESS_KEY_SECRET")

In [2]:
import boto3
import json

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client(
    "bedrock-agent-runtime", 
    region_name="us-east-1",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=YOUR_SECRET_ACCESS_KEY,
)


In [ ]:
# # Set the model ID, e.g., Llama 3 8b Instruct.
# model_id = "meta.llama3-8b-instruct-v1:0"

In [3]:
agentId="80OBUFCZVY"
agentAliasId="DB3APGS6J1",
sessionId="d244aa5e-0c51-44cf-ab6b-64ff2c5b8452"

In [16]:

def invoke_agent(self, agent_id, agent_alias_id, session_id, prompt):
    """
    Sends a prompt for the agent to process and respond to.

    :param agent_id: The unique identifier of the agent to use.
    :param agent_alias_id: The alias of the agent to use.
    :param session_id: The unique identifier of the session. Use the same value across requests
                        to continue the same conversation.
    :param prompt: The prompt that you want Claude to complete.
    :return: Inference response from the model.
    """

    try:
        # Note: The execution time depends on the foundation model, complexity of the agent,
        # and the length of the prompt. In some cases, it can take up to a minute or more to
        # generate a response.
        response = self.agents_runtime_client.invoke_agent(
            agentId=agent_id,
            agentAliasId=agent_alias_id,
            sessionID=session_id,
            inputText=prompt
        )

        completion = ""

        for event in response.get("completion"):
            chunk = event["chunk"]
            completion = completion + chunk["bytes"].decode()

    except ClientError as e:
        logger.error(f"Couldn't invoke agent. {e}")
        raise

    return completion




In [17]:
response = client.invoke_agent(
    agentId="80OBUFCZVY",
    agentAliasId="DB3APGS6J1",
    enableTrace=False,
    endSession=False,
    inputText='I feel sad',
    sessionId="d244aa5e-0c51-44cf-ab6b-64ff2c5b8452",
    sessionState={
        'invocationId': '9789c702-f9da-49d1-aeaa-d4a527da6d57',
    }
)

In [9]:
response

{'ResponseMetadata': {'RequestId': '6ef0bcc1-5335-4e99-8651-62fb044d5f32',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 09 Jul 2024 12:51:23 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6ef0bcc1-5335-4e99-8651-62fb044d5f32',
   'x-amz-bedrock-agent-session-id': 'd244aa5e-0c51-44cf-ab6b-64ff2c5b8452',
   'x-amzn-bedrock-agent-content-type': 'application/json'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'sessionId': 'd244aa5e-0c51-44cf-ab6b-64ff2c5b8452',
 'completion': <botocore.eventstream.EventStream at 0x7f33f5c08b30>}

In [20]:
completion = ""
for event in response.get("completion"):
    chunk = event["chunk"]
    completion += chunk["bytes"].decode()

print("Agent's response:", completion)

Agent's response: I'm sorry to hear you are feeling sad. It's understandable to feel that way sometimes. Here are some things that may help lift your mood:

Doing activities you enjoy, going for a walk, getting exercise, and spending time with people you care about. Relaxation techniques like yoga or meditation could help too. 

I hope those give you some ideas. Please let me know if you need any other suggestions for coping with sadness. I'm here to listen and provide support.


In [ ]:
import json
import boto3

def lambda_handler(event, context):
    client = boto3.client(
    "bedrock-agent-runtime", 
    region_name="us-east-1",
    )
    
    response = client.invoke_agent(
    agentId="80OBUFCZVY",
    agentAliasId="DB3APGS6J1",
    enableTrace=False,
    endSession=False,
    inputText='I feel sad',
    sessionId="d244aa5e-0c51-44cf-ab6b-64ff2c5b8452",
    sessionState={
        'invocationId': '9789c702-f9da-49d1-aeaa-d4a527da6d57',
        }
    )
    
    completion = ""
    for event in response.get("completion"):
        chunk = event["chunk"]
        completion += chunk["bytes"].decode()
        
    return {
        'statusCode': 200,
        'body': "Agent's response:" + completion
    }

In [1]:
# last version with gateway
#Gateway HHP API -> improved like rawPath: "/conversation"
# body {json}
# \"message\" : \"your text\"

import json
import boto3

def lambda_handler(event, context):
    client = boto3.client(
    "bedrock-agent-runtime", 
    region_name="us-east-1",
    )
    
    
    body = event.get('body', '{}')
    try:
        parsed_body = json.loads(body)
    except json.JSONDecodeError:
        parsed_body = {}
    
    response = client.invoke_agent(
        agentId="80OBUFCZVY",
        agentAliasId="DB3APGS6J1",
        enableTrace=False,
        endSession=False,
        inputText=parsed_body['message'],
        sessionId="d244aa5e-0c51-44cf-ab6b-64ff2c5b8452",
    )
    
    
    completion = ""
    for event in response.get("completion"):
        chunk = event["chunk"]
        completion += chunk["bytes"].decode()
        
        
    return {
        'statusCode': 200,
        'body': "Agent's response:" + completion
    }